Calcular descriptors moleculares 1D, 2D y 3D con los programas e-Dragon y PaDEL a los siguientes compuestos. Tenga en cuenta que las moléculas se encuentran en 3D y debe convertirlas a 2D, puede utilizar el software OpenBabel

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem

def convert_2d_to_3d(archivo2d):
    """
    Convierte un archivo .mol (2D o sin coordenadas) a 3D,
    optimizando con MMFF94.
    """
    # Leer la molécula (puede ser 2D o sin coordenadas)
    mol2d = Chem.MolFromMolFile(archivo2d, removeHs=False)
    
    if mol2d is None:
        print("❌ No se pudo leer el archivo:", archivo2d)
        return None
    
    # Añadir hidrógenos explícitos (importante para 3D)
    mol3d = Chem.AddHs(mol2d)
    
    # Generar conformación 3D
    if AllChem.EmbedMolecule(mol3d, randomSeed=42) == -1:
        print("❌ No se pudo generar la conformación 3D.")
        return None
    
    # Optimizar geometría con MMFF94 (requiere hidrógenos)
    try:
        AllChem.MMFFOptimizeMolecule(mol3d)
    except ValueError:
        # Si MMFF falla, intentar con UFF
        print("⚠️ MMFF no disponible, usando UFF...")
        AllChem.UFFOptimizeMolecule(mol3d)
    
    # Guardar archivo 3D (con hidrógenos)
    archivo3d = archivo2d.replace('.mol', '_3d.mol')
    Chem.MolToMolFile(mol3d, archivo3d)
    
    print("✅ Convertido a 3D:", archivo3d)
    return archivo3d

# === Uso ===
archivo2d = '../data/Antonio.mol'  # puede ser 2D o incluso sin coordenadas
archivo3d = convert_2d_to_3d(archivo2d)

✅ Convertido a 3D: ../data/Antonio_3d.mol


In [3]:
from rdkit import Chem
from rdkit.Chem import Draw
import os

# Ruta al archivo .mol
archivo_mol = archivo3d

# Verifica que el archivo exista
if not os.path.exists(archivo_mol):
    raise FileNotFoundError(f"No se encontró el archivo: {archivo_mol}")

# Cargar la molécula desde el archivo .mol
mol = Chem.MolFromMolFile(archivo_mol, sanitize=True, removeHs=False)

# Verificar si se cargó correctamente
if mol is None:
    raise ValueError("No se pudo cargar la molécula. Revisa el formato del archivo .mol.")

# Mostrar en 2D (RDKit genera una representación 2D bonita)
Draw.MolToImage(mol, size=(600, 600)).show()

# Opcional: guardar la imagen en lugar de mostrarla
# Draw.MolToFile(mol, "molecula.png", size=(600, 600))

In [6]:
from rdkit import Chem
import py3Dmol

# Cargar la molécula desde el archivo .mol
mol = Chem.MolFromMolFile(archivo_mol, removeHs=False, sanitize=True)

if mol is None:
    raise ValueError("No se pudo cargar la molécula. Revisa el archivo .mol.")

# Convertir a formato MOL para py3Dmol
mol_block = Chem.MolToMolBlock(mol)

# Crear la vista 3D
view = py3Dmol.view(width=600, height=600)
view.addModel(mol_block, 'mol')
view.setStyle({'stick': {}})  # Puedes usar 'sphere', 'stick', 'line', etc.
view.setBackgroundColor('white')
view.zoomTo()

# Mostrar en Jupyter (si estás en notebook)
view.show()

# Guardar y abrir en navegador (funciona en cualquier entorno)
#view.save('molecula_3d.html')
#print("Visualización 3D guardada como 'molecula_3d.html'. Ábrela en tu navegador.")

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [2]:
import os
import tempfile
from padelpy import padeldescriptor
import pandas as pd

# Ruta a tu molécula 3D
mol_file = '../data/Antonio_3d.mol'

# Verificar que exista
if not os.path.exists(mol_file):
    raise FileNotFoundError(f"Archivo no encontrado: {mol_file}")

# Crear un archivo temporal con la lista de moléculas
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.txt') as f:
    f.write(os.path.abspath(mol_file) + '\n')
    list_file = f.name

try:
    # Ejecutar PaDEL
    padeldescriptor(
        mol_dir=list_file,
        d_file=None,  # No guardar a disco, devolver dict
        d_2d=True,
        d_3d=True,
        fingerprints=True,
        removesalt=False,
        standardizenitro=False,
        log=False
    )
    
    # Pero... ¡padeldescriptor no devuelve nada si d_file=None!
    # Así que mejor: guardar a archivo temporal y leerlo

    temp_csv = list_file.replace('.txt', '.csv')
    padeldescriptor(
        mol_dir=list_file,
        d_file=temp_csv,
        d_2d=True,
        d_3d=True,
        fingerprints=True,
        removesalt=False,
        standardizenitro=False,
        log=False
    )
    
    # Leer el CSV
    df = pd.read_csv(temp_csv, index_col=0)
    
    # Mostrar resultados
    print("✅ Descriptores calculados:")
    print(f"Forma: {df.shape}")
    print("\nPrimeras columnas:")
    print(df.iloc[:, :5].T)
    
    # Guardar si lo deseas
    # df.to_csv("../data/resultPadel.csv")
    
finally:
    # Limpiar archivos temporales
    for f in [list_file, list_file.replace('.txt', '.csv')]:
        if os.path.exists(f):
            os.remove(f)

✅ Descriptores calculados:
Forma: (0, 2756)

Primeras columnas:
Empty DataFrame
Columns: []
Index: [nAcid, ALogP, ALogp2, AMR, apol]


In [7]:
import os
import tempfile
from rdkit import Chem
from padelpy import padeldescriptor
import pandas as pd

# 1. Leer molécula y obtener SMILES
mol_file = '../data/Antonio_3d.mol'
mol = Chem.MolFromMolFile(mol_file, removeHs=False)
if mol is None:
    raise ValueError("No se pudo leer el archivo .mol")

# Canonical SMILES (sin coordenadas, solo conectividad)
smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
name = os.path.splitext(os.path.basename(mol_file))[0]

print(f"SMILES: {smiles}")

# 2. Crear archivo SMILES temporal
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.smi') as f:
    f.write(f"{smiles}\t{name}\n")
    smi_file = f.name

# 3. Archivo CSV temporal
temp_csv = smi_file.replace('.smi', '.csv')

try:
    # 4. Ejecutar PaDEL con SMILES
    padeldescriptor(
        mol_dir=smi_file,
        d_file=temp_csv,
        d_2d=True,
        d_3d=True,
        fingerprints=False,
        removesalt=False,
        standardizenitro=False,
        log=False
    )
    
    # 5. Leer resultados
    df = pd.read_csv(temp_csv, index_col=0)
    print(f"\n✅ Resultado: {df.shape[0]} moléculas, {df.shape[1]} descriptores")
    
    if df.shape[0] > 0:
        print("\nPrimeras filas:")
        print(df.head())
        # df.to_csv("../data/resultPadel.csv")
    else:
        print("\n❌ ¡Aún vacío! Revisa PaDEL.log")
        if os.path.exists("PaDEL.log"):
            with open("PaDEL.log") as logf:
                print("Log:", logf.read()[-500:])  # últimas 500 letras

finally:
    # Limpiar
    for f in [smi_file, temp_csv]:
        if os.path.exists(f):
            os.remove(f)

SMILES: [H]OC(=O)C([H])([H])N([H])c1c([H])c([H])c([H])c(-c2c([H])c([H])c([H])c([H])c2-c2nc(-c3c([H])c([H])c([H])c([H])c3[H])c(-c3c([H])c([H])c([H])c([H])c3[H])n2[H])c1[H]

✅ Resultado: 1 moléculas, 1875 descriptores

Primeras filas:
            nAcid   ALogP   ALogp2      AMR       apol  naAromAtom  nAromBond  \
Name                                                                            
Antonio_3d      1 -1.2824  1.64455  18.3813  71.280239          29         33   

            nAtom  nHeavyAtom  nH  ...  P1s  P2s  E1s  E2s  E3s  Ts  As  Vs  \
Name                               ...                                        
Antonio_3d     57          34  23  ...  NaN  NaN  NaN  NaN  NaN NaN NaN NaN   

            Ks  Ds  
Name                
Antonio_3d NaN NaN  

[1 rows x 1875 columns]


In [4]:
df.to_csv('../data/pydelDEscript.csv', index=False)

## Comparar

In [5]:
import pandas as pd

# Cargar ambos archivos
df_padel = pd.read_csv("../data/molecules3.csv", index_col=0)
df_padelpy = pd.read_csv("../data/pydelDEscript.csv", index_col=0)

In [6]:
df_padelpy.insert(0, "Name", "Mol1")
df_padelpy.to_csv("../data/pydelDEscript.csv", index=False)

print(f"✅ Añadida columna 'Name' ")

✅ Añadida columna 'Name' 


In [7]:
print("PaDEL shape:", df_padel.shape)
print("padelpy shape:", df_padelpy.shape)

PaDEL shape: (1, 1875)
padelpy shape: (1, 1875)


In [8]:
# Alinear columnas
common_cols = df_padel.columns.intersection(df_padelpy.columns)
df_padel = df_padel[common_cols]
df_padelpy = df_padelpy[common_cols]

# Alinear índice (nombre de molécula)
df_padel = df_padel.sort_index()
df_padelpy = df_padelpy.sort_index()

In [9]:
df_padel.head()

,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,nB,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
Name,,,,,,,,,,,,,,,,,,,,,
AUTOGEN_Antonio,-1.2824,1.64455,18.3813,71.280239,29,33,57,34,23,0,...,0.52829,0.350703,0.442626,0.383121,0.417171,18.119421,95.748456,247.237328,0.318489,1.242918


In [10]:
df_padelpy.head()

,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,nB,...,P1s,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds
nAcid,,,,,,,,,,,,,,,,,,,,,
1,-1.2824,1.64455,18.3813,71.280239,29,33,57,34,23,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_padel_num = df_padel.drop(columns=['Name'])
df_padelpy_num = df_padelpy.drop(columns=['nAcid'])

In [11]:
# Calcular diferencias
diff = (df_padel - df_padelpy).abs()

# Estadísticas
print("\nDiferencias:")
print("Máxima diferencia:", diff.max().max())
print("Número de valores diferentes (>1e-6):", (diff > 1e-6).sum().sum())


Diferencias:
Máxima diferencia: nan
Número de valores diferentes (>1e-6): 0


In [12]:
# Mostrar descriptores con mayor diferencia
max_diff_desc = diff.stack().sort_values(ascending=False).head(5)
print("\nTop 5 descriptores con mayor diferencia:")
print(max_diff_desc)


Top 5 descriptores con mayor diferencia:
Series([], dtype: float64)


In [13]:
are_equal = df_padel.equals(df_padelpy)
print("\n¿Son idénticos?", are_equal)

if not are_equal:
    print("⚠️ Hay diferencias numéricas (posiblemente por redondeo o versión de PaDEL)")
else:
    print("✅ ¡Ambos métodos dan resultados idénticos!")


¿Son idénticos? False
⚠️ Hay diferencias numéricas (posiblemente por redondeo o versión de PaDEL)


In [12]:
import pandas as pd

# Cargar el archivo
df = pd.read_csv("../data/vcc.txt", sep="\t", skiprows=2)  # La primera línea es encabezado
# Limpiar espacios en los nombres de columna
df.columns = df.columns.str.strip()
# Ver las primeras columnas y filas
print(df.head())
print(df.columns.tolist()[:20])  # Ver primeros 20 nombres de descriptores

   No.    MOL_ID     MW   AMW    Sv     Se    Sp    Ss    Mv    Me  ...  \
0    1       CCC  44.11  4.01  5.39  10.53  6.05   5.5  0.49  0.96  ...   
1    2  c1ccccc1  78.12  6.51  7.79  11.65  8.28  12.0  0.65  0.97  ...   

   Hypnotic-50  Neoplastic-80  Neoplastic-50  Infective-80  Infective-50  \
0            0              0              0             0             0   
1            0              0              0             0             0   

   BLTF96  BLTD48  BLTA96  ALOGPS_logP  ALOGPS_logS  
0   -3.32   -3.48   -3.51         2.19        -1.15  
1   -3.31   -3.46   -3.48         2.03        -1.86  

[2 rows x 1668 columns]
['No.', 'MOL_ID', 'MW', 'AMW', 'Sv', 'Se', 'Sp', 'Ss', 'Mv', 'Me', 'Mp', 'Ms', 'nAT', 'nSK', 'nBT', 'nBO', 'nBM', 'SCBO', 'ARR', 'nCIC']


In [13]:
df.replace(-999, pd.NA, inplace=True)
df.dropna(axis=1, how='all', inplace=True)  # elimina columnas sin datos

In [14]:
# Comparar peso molecular y número de donadores de H
print(df[["MOL_ID", "MW", "nHDon", "nHAcc"]])

     MOL_ID     MW  nHDon  nHAcc
0       CCC  44.11      0      0
1  c1ccccc1  78.12      0      0
